## Create a generative AI runbook to resolve security findings

## Module 2

```
// Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
// SPDX-License-Identifier: MIT-0
```

### Introduction

In this module, you will learn the art-of-the-possible and dive deep into some of the advanced features of Claude 3.
- Create a tool for Claude 3 to use
- Create a function to support when the tool is used
- Pass the results of the tool back into Claude 3
- Get the final result

**NOTE**: You can use SHIFT + ENTER to run each of the cells in this workbook.

With tool use, we substitute tool or function results into prompts. Claude 3 can't literally call or access tools and functions. Instead, we have Claude 3:

- Output the tool name and arguments it wants to call
- Halt any further response generation while the tool is called
- Then we reprompt with the appended tool results

Function calling is useful because it expands Claude 3's capabilities and enables Claude 3 to handle much more complex, multi-step tasks. 

We will use tool use / function calling to expand Claude 3's knowledge of Security Hub findings.

#### Step 1: Create a tool for Claude 3 to use

In this step, we will setup our function to access Claude 3 and create a tool.

In [ ]:
%pip install anthropic --quiet

import os
import sys
import re
from anthropic import AnthropicBedrock

MODEL_NAME='anthropic.claude-3-sonnet-20240229-v1:0'
AWS_REGION = "us-west-2"

client = AnthropicBedrock(aws_region=AWS_REGION)

# Rewrittten to call Claude 3 Sonnet, which is generally better at tool use, and include stop_sequences
def get_completion(messages, system_prompt="", prefill="",stop_sequences=None):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages,
        system=system_prompt,
        stop_sequences=stop_sequences
    )
    return message.content[0].text

The system prompt gives context to the tools Claude 3 has access to. In this workshop, we are using a similar prompt from the Claude 3 documentation.

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of functions you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

You can invoke one or more functions by writing a "<function_calls>" block like the following as part of your
reply to the user:
<function_calls>
<invoke name="$FUNCTION_NAME">
<antml:parameter name="$PARAMETER_NAME">$PARAMETER_VALUE</parameter>
...
</invoke>
<invoke name="$FUNCTION_NAME2">
...
</invoke>
</function_calls>

String and scalar parameters should be specified as is, while lists and objects should use JSON format. Note that
spaces for string values are not stripped. The output is not expected to be valid XML and is parsed with regular
expressions.

The output and/or any errors will appear in a subsequent "<function_results>" block, and remain there as part of
your reply to the user.
You may then continue composing the rest of your reply to the user, respond to any errors, or make further function
calls as appropriate.
If a "<function_results>" does NOT appear after your function calls, then they are likely malformatted and not
recognized as a call."""

In the specific tool prompt, we define a function that Claude 3 has access to. 

The function is called `get_findings_by_severity` and takes two parameters:
- `severity` - the severity of the findings, which could be CRITICAL, HIGH or MEDIUM.
- `Limit` - the number of findings to return.

In [ ]:
system_prompt_tools_specific_tools = """Here are the functions available in JSONSchema format:
<tools>
<tool_description>
<tool_name>get_findings_by_severity</tool_name>
<description>
Lookup function for getting findings from Security Hub.
Supports CRITICAL, HIGH, and MEDIUM severity.
</description>
<parameters>
<parameter>
<name>severity</name>
<type>String</type>
<description>Severity level of the findings to return. This should be CRITICAL, HIGH, or MEDIUM. They must be uppercase</description>
</parameter>
<parameter>
<name>limit</name>
<type>int</type>
<description>This parameter determine how many results to return</description>
</parameter>
</parameters>
</tool_description>
</tools>
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

We then update our prompt. In our example, we want to find 3 medium findings in Security Hub.

In [ ]:
lookup_prompt = {
    "role": "user",
    "content": "What are 3 medium findings in Security Hub?"
}

stop_sequences = ["</function_calls>"]

# Get Claude 3's response
function_calling_response = get_completion([lookup_prompt], system_prompt=system_prompt, stop_sequences=stop_sequences)
print(function_calling_response)

#### Step 2: Create a function to support when the tool is used

Claude 3 outputed that it would like to use the new tool that we created `get_findings_by_severity`. In this step we will define it.

Just like in Module 1, we use boto3 to make an API call to Security Hub.

In [ ]:
import boto3
sh = boto3.client('securityhub')

def get_findings_by_severity(severity, limit):
    response = sh.get_findings(
        Filters={
                'SeverityLabel': [
                    {
                        'Value': severity,
                        'Comparison': 'EQUALS'
                    }
                ]
        },
        MaxResults=limit
    )
    return response['Findings']

The parameters Claude 3 would like to use, are wrapped in tags:

```
<parameter name="severity">MEDIUM</parameter>
<parameter name="limit">3</parameter>
```

Next, use the function below to extract them and call the `getFindingsBySeverity` function.

In [ ]:
# Extract the paremters from Claude 3 and call our new function

def find_parameter(message, parameter_name):
    parameter_start_string = f"name=\"{parameter_name}\">"
    start = message.index(parameter_start_string)
    if start == -1:
        return None
    if start > 0:
        start = start + len(parameter_start_string)
        end = start
        while message[end] != "<":
            end += 1
    return message[start:end]

severity = find_parameter(function_calling_response, "severity")
limit = find_parameter(function_calling_response, "limit")

if severity and limit:
    result = get_findings_by_severity(str(severity), int(limit))
    print("---------------- RESULT ----------------")
    print(f"{result}")

#### Step 3: Pass the results of the tool back into Claude 3

The output of the function is a JSON representation of the findings, just like we got in module 1. Claude 3 expects the result of the function to be formatted in a specific format. Run the following function to format it.

In [ ]:
# Now we need to properly format the results, so we can pass back into Claude 3

def construct_successful_function_run_injection_prompt(invoke_results):
    constructed_prompt = (
        "<function_results>\n"
        + '\n'.join(
            f"<result>\n<tool_name>{res['tool_name']}</tool_name>\n<stdout>\n{res['tool_result']}\n</stdout>\n</result>"
            for res in invoke_results
        ) + "\n</function_results>"
    )

    return constructed_prompt

formatted_results = [{
    'tool_name': 'get_findings_by_severity',
    'tool_result': result
}]
function_results = construct_successful_function_run_injection_prompt(formatted_results)
print(function_results)


#### Step 4: Get the final result

In this step, we take the formatted findings, and pass them back into Claude 3 to get our final result.

In [ ]:
# Send back to Claude 3 by appending the result back to the message chain

full_first_response = function_calling_response + "</function_calls>"

# Construct the full conversation
messages = [lookup_prompt,
{
    "role": "assistant",
    "content": full_first_response
},
{
    "role": "user",
    "content": function_results
}]
   
# Print Claude 3's response
final_response = get_completion(messages, system_prompt=system_prompt, stop_sequences=stop_sequences)
print("------------- FINAL RESULT -------------")
print(final_response)